In [3]:
import pandas as pd

df = pd.read_csv('/home/vivek/Desktop/INOP/data/Quote-Equity-JIOFIN-EQ-30-03-2024-to-30-03-2025.csv')

In [7]:
# Strip extra spaces from column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Convert 'date' to datetime format
df['date'] = pd.to_datetime(df['date'])

# Sort data by date
df = df.sort_values(by="date").reset_index(drop=True)

# Display cleaned columns
print(df.head())

        date series   open    high     low  prev._close     ltp   close  \
0 2024-04-01     EQ  357.0  364.20  355.60       353.75  357.90  356.95   
1 2024-04-02     EQ  359.0  362.85  356.50       356.95  357.95  357.10   
2 2024-04-03     EQ  356.5  359.80  353.20       357.10  357.00  357.10   
3 2024-04-04     EQ  359.4  367.80  358.00       357.10  362.20  361.30   
4 2024-04-05     EQ  361.0  374.85  359.05       361.30  373.00  373.20   

     vwap   52w_h   52w_l       volume               value no_of_trades  
0  358.96  374.50  204.25  2,36,44,472   8,48,74,85,998.45     1,84,560  
1  359.04  374.50  204.25  2,96,84,726  10,65,81,25,059.05     1,77,541  
2  356.65  374.50  204.25  1,79,20,490   6,39,13,43,231.25     1,40,410  
3  362.67  374.50  204.25  3,39,26,426  12,30,40,97,378.45     2,12,104  
4  369.34  374.85  204.25  4,69,79,676  17,35,13,39,873.25     2,96,698  


In [9]:
df['daily_return'] = df['close'].pct_change()

In [10]:
expected_mean_return = df['daily_return'].mean()
print(f"Expected Mean Return: {expected_mean_return:.5f}")


Expected Mean Return: -0.00160


In [11]:
risk = df['daily_return'].std()
print(f"Stock Risk (Volatility): {risk:.5f}")


Stock Risk (Volatility): 0.02056


In [12]:
risk_free_rate = 0.0002  # Assuming 5% annual return

sharpe_ratio = (expected_mean_return - risk_free_rate) / risk
print(f"Sharpe Ratio: {sharpe_ratio:.5f}")


Sharpe Ratio: -0.08776


In [13]:
df['cumulative_return'] = (1 + df['daily_return']).cumprod()


In [14]:
df['sma_50'] = df['close'].rolling(window=50).mean()
df['sma_200'] = df['close'].rolling(window=200).mean()
df['volatility_20'] = df['daily_return'].rolling(window=20).std()


In [15]:
df['rolling_max'] = df['close'].cummax()
df['drawdown'] = df['close'] / df['rolling_max'] - 1
max_drawdown = df['drawdown'].min()
print(f"Max Drawdown: {max_drawdown:.5f}")


Max Drawdown: -0.48210


In [16]:
import numpy as np

def compute_rsi(data, window=14):
    delta = data.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

df['rsi_14'] = compute_rsi(df['close'])


In [17]:
df['bollinger_upper'] = df['sma_50'] + 2 * df['volatility_20']
df['bollinger_lower'] = df['sma_50'] - 2 * df['volatility_20']


In [19]:
# # Load Nifty 50 index data
# df_nifty = pd.read_csv("nifty50_data.csv")
# df_nifty.columns = df_nifty.columns.str.strip().str.lower().str.replace(" ", "_")

# # Merge stock and index data
# df_merged = df.merge(df_nifty[['date', 'close']], on='date', suffixes=('', '_nifty'))

# # Compute correlation (Beta)
# beta = df_merged['daily_return'].corr(df_merged['close_nifty'].pct_change())
# print(f"Stock Beta: {beta:.5f}")
